In [4]:
#Load data, filter, make folders
import numpy as np
import plotly.graph_objects as go
import pandas as pd

from main_meg_qc import initial_stuff

In [5]:
duration=1
n_events, df_epochs_mags, df_epochs_grads, epochs_mags, epochs_grads, mags, grads, filtered_d, filtered_d_resamp, raw_cropped, raw=initial_stuff(duration)

Opening raw data file ../data/sub_HT05ND16/210811/mikado-1.fif/...
    Read a total of 8 projection items:
        magn8_iasoff_68deg.fif : PCA-v1 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v2 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v3 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v4 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v5 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v6 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v7 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v8 (1 x 306)  idle
    Range : 1809000 ... 3375999 =   1809.000 ...  3375.999 secs
Ready.
Opening raw data file /Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/Functions/../data/sub_HT05ND16/210811/mikado-2.fif...
    Read a total of 8 projection items:
        magn8_iasoff_68deg.fif : PCA-v1 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v2 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v3 (1 x 306)  idle
        magn8_iasoff_68deg.fif :

/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/Functions/data_load_and_folders.py:8: RuntimeWarning: This filename (../data/sub_HT05ND16/210811/mikado-1.fif/) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(raw_file)


Reading 0 ... 60000  =      0.000 ...    60.000 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 1e+02 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 0.50, 100.00 Hz: -6.02, -6.02 dB

Trigger channel has a non-zero initial value of 18 (consider using initial_event=True to detect this event)
50 events found
Event IDs: [    9    19    21    23    27    31 16393 16403 16405 16411 32749 32750
 32759]


/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/Functions/data_load_and_folders.py:65: RuntimeWarning: Trigger channel contains negative values, using absolute value. If data were acquired on a Neuromag system with STI016 active, consider using uint_cast=True to work around an acquisition bug
  raw_bandpass_resamp.resample(sfreq=h_freq*5)


Trigger channel has a non-zero initial value of 18 (consider using initial_event=True to detect this event)
42 events found
Event IDs: [    9    19    20    21    22    23    27    31 16393 16402 16403 16405
 16411 32741 32749 32750 32759]
Trigger channel has a non-zero initial value of 18 (consider using initial_event=True to detect this event)
5 events found
Event IDs: [ 9 19 20 21]
Not setting metadata
Not setting metadata
5 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 8)
8 projection items activated
Loading data for 5 events and 1201 original time points ...
0 bad epochs dropped
Not setting metadata
Not setting metadata
5 matching events found
No baseline correction applied
8 projection items activated
Loading data for 5 events and 1201 original time points ...
0 bad epochs dropped


/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/Functions/data_load_and_folders.py:65: RuntimeWarning: Trigger channel contains negative values, using absolute value. If data were acquired on a Neuromag system with STI016 active, consider using uint_cast=True to work around an acquisition bug
  raw_bandpass_resamp.resample(sfreq=h_freq*5)
/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/Functions/data_load_and_folders.py:80: RuntimeWarning: Trigger channel contains negative values, using absolute value. If data were acquired on a Neuromag system with STI016 active, consider using uint_cast=True to work around an acquisition bug
  events = mne.find_events(data, stim_channel=stim_channel, min_duration=event_dur)


In [33]:
# Separate data for mags and grads in 2 arrays.

selected_mags = [item[1] for item in mags]
selected_grads = [item[1] for item in grads]
data_mags, _ = filtered_d_resamp[selected_mags, :]  
data_grads, _ = filtered_d_resamp[selected_grads, :]  

dt=duration/len(data_mags[0]) #samplig frequency (resampled data)

In [7]:
#EXAMPLE: find peaks of the first magnetometer channel:

from mne.preprocessing import peak_finder

thresh=(max(data_mags[0]) - min(data_mags[0])) / 1.7 #THRESHOLD FOR IDENTIFICATION ADJUST HERE
# The amount above surrounding data for a peak to be identified. Larger values mean the algorithm 
# is more selective in finding peaks. If None, use the default of (max(x0) - min(x0)) / 4.

peak_locs, peak_magnitudes = peak_finder(data_mags[0], extrema=1, thresh=thresh) #positive peaks
neg_peak_locs, neg_peak_magnitudes = peak_finder(data_mags[0], extrema=-1, thresh=thresh) #negative peaks

t=np.arange(0, duration, dt) 
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=data_mags[0], name='data'));
fig.add_trace(go.Scatter(x=t[peak_locs], y=peak_magnitudes, mode='markers', name='+peak'));
fig.add_trace(go.Scatter(x=t[neg_peak_locs], y=neg_peak_magnitudes, mode='markers', name='-peak'));
fig.update_layout(
    title={
    'text': "First magnetometer: peaks detected",
    'y':0.85,
    'x':0.5,
    'xanchor': 'center',
    'yanchor': 'top'})
    
fig.show()


#https://www.codegrepper.com/code-examples/python/go.Scatter+no+line

Found 13 significant peaks
Found 12 significant peaks


In [8]:
#EXAMPLE part2: Now calculate mean of the neighbouring peaks and plot it:

pair_dist_sec=0.02
pair_dist=pair_dist_sec/dt
pair_dist

pairs_magnitudes=[]
pairs_locs=[]

for posit_pair_ind, posit_pair in enumerate(peak_locs):
    neg_pair_ind=np.where(np.logical_and(neg_peak_locs>=posit_pair-pair_dist/2, neg_peak_locs<=posit_pair+pair_dist/2))
    if neg_pair_ind[0].size != 0:
        pairs_locs.append([peak_locs[posit_pair_ind], neg_peak_locs[neg_pair_ind[0][0]]])
        pairs_magnitudes.append([peak_magnitudes[posit_pair_ind], neg_peak_magnitudes[neg_pair_ind[0][0]]])

amplitude=np.zeros(len(pairs_magnitudes),)
for i, pair in enumerate(pairs_magnitudes):
    print(pair)
    amplitude[i]=pair[0]-pair[1]

mean_amplitude=np.mean(amplitude)


#This for plotting only:
list_pairs_locs = [i for sub in pairs_locs for i in sub]
list_pairs_magnitudes = [i for sub in pairs_magnitudes for i in sub]


fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=data_mags[0], name='data'));
fig.add_trace(go.Scatter(x=t[list_pairs_locs], y=list_pairs_magnitudes, mode='markers', name='pair'));
fig.update_layout(
    title={
    'text': "First magnetometer: PAIRS of peaks detected",
    'y':0.85,
    'x':0.5,
    'xanchor': 'center',
    'yanchor': 'top'})
    
fig.show()

print(amplitude)
print('mean: ', mean_amplitude)


[1.2750223727985316e-12, -2.348914740882849e-12]
[1.6536776681718568e-12, -2.7007423152539433e-12]
[9.981386805213335e-13, -2.3043926878575423e-12]
[1.843137032278871e-12, -2.2134551974137203e-12]


[3.62393711e-12 4.35441998e-12 3.30253137e-12 4.05659223e-12]
mean:  3.834370173794662e-12


In [47]:
def neighbour_peak_amplitude(pair_dist_sec: float, dt: float, pos_peak_locs:np.ndarray, neg_peak_locs:np.ndarray, pos_peak_magnitudes: np.ndarray, neg_peak_magnitudes: np.ndarray) -> float:

    ''' Function finds a pair: postive+negative peak and calculates the amplitude between them. 
    If no neighbour is found withing given distance - this peak is skipped. 
    If several neighbours are found - several pairs are created. '''


    pair_dist=pair_dist_sec/dt
    pairs_magnitudes=[]
    pairs_locs=[]

    for posit_pair_ind, posit_pair in enumerate(pos_peak_locs):
        neg_pair_ind=np.where(np.logical_and(neg_peak_locs>=posit_pair-pair_dist/2, neg_peak_locs<=posit_pair+pair_dist/2))
        if neg_pair_ind[0].size != 0:
            pairs_locs.append([pos_peak_locs[posit_pair_ind], neg_peak_locs[neg_pair_ind[0][0]]])
            pairs_magnitudes.append([pos_peak_magnitudes[posit_pair_ind], neg_peak_magnitudes[neg_pair_ind[0][0]]])

    amplitude=np.zeros(len(pairs_magnitudes),)
    for i, pair in enumerate(pairs_magnitudes):
        amplitude[i]=pair[0]-pair[1]

    return np.mean(amplitude)


In [48]:
def peak_amplitude_per_epoch(mg_names: list, df_epoch_mg: pd.DataFrame, dt: float, n_events: int, thresh_lvl: float, pair_dist_sec: float) -> pd.DataFrame:

    '''Function loops over mags or grads (separate epochs) and calculates peak-to-peak amplitude'''
    
    dt=duration/len(data_mags[0])
    
    dict_mg = {}

    mg_only_names=[n[0] for n in mg_names]

    epoch_numbers=[ep for ep in range(n_events)]
    for ep in epoch_numbers: #loop over each epoch
        rows_for_ep = [row for row in df_epoch_mg.iloc if row.epoch == ep] #take all rows of 1 epoch, all channels.
  
        peak_ampl_epoch=[]
        for ch_name in mg_only_names: #loop over channel names
            data_ch_epoch = [row_mg[ch_name] for row_mg in rows_for_ep] #take the data for 1 epoch for 1 channel

            thresh=(max(data_ch_epoch) - min(data_ch_epoch)) / thresh_lvl 
            #can also change the whole thresh to a single number setting

            pos_peak_locs, pos_peak_magnitudes = peak_finder(data_ch_epoch, extrema=1, thresh=thresh, verbose=False) #positive peaks
            neg_peak_locs, neg_peak_magnitudes = peak_finder(data_ch_epoch, extrema=-1, thresh=thresh, verbose=False) #negative peaks
            
            pp_ampl=neighbour_peak_amplitude(pair_dist_sec, dt, pos_peak_locs, neg_peak_locs, pos_peak_magnitudes, neg_peak_magnitudes)
            peak_ampl_epoch.append(pp_ampl)

        dict_mg[ep] = peak_ampl_epoch

    df_pp_ampl_mg = pd.DataFrame(dict_mg, index=mg_only_names)

    return(df_pp_ampl_mg)

In [45]:
df_pp_ampl_mags=peak_amplitude_per_epoch(mg_names=mags, df_epoch_mg=df_epochs_mags, dt=dt, n_events=n_events, thresh_lvl=2, pair_dist_sec=0.02)


In [37]:
#Copied from rmse file. once it s saved as py file, can just call this functions

def boxplot_std_channel_epoch_hovering_plotly(df_mg: pd.DataFrame, tit: str, sid: str):

  unit='?'
  if tit=='Magnetometers':
    unit='T'
  elif tit=='Gradiometers':
    unit='T/m'
  else:
    print('Please check tit input. Has to be "Magnetoneters" or "Gradiometers"')

  #transpose the data to plot the boxplots on x axes
  df_mg_transposed = df_mg.T 

  #collect all names of original df into a list to use as tick labels:
  ch_names=list(df_mg_transposed) 

  fig = go.Figure()

  for col in df_mg_transposed:
    fig.add_trace(go.Box(y=df_mg_transposed[col].values, 
      name=df_mg_transposed[col].name, 
      opacity=0.7, 
      boxpoints="all", 
      pointpos=0,
      marker_size=3,
      line_width=1,
      text=df_mg_transposed[col].index))
    fig.update_traces(hovertemplate='Epoch: %{text}'+'<br>Amplitude: %{y: .0f}')
      

  fig.update_layout(
      xaxis = dict(
          tickmode = 'array',
          tickvals = [v for v in range(0, len(ch_names))],
          ticktext = ch_names,
          rangeslider=dict(visible=True)
      ),
      yaxis = dict(
        showexponent = 'all',
        exponentformat = 'e'),
      yaxis_title="Peak-to-peak amplitude in "+unit,
      title={
        'text': "Peak-to-peak amplitude over epochs for "+tit,
        'y':0.85,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
    
  fig.show()

  fig_name='PP_amplitude_epochs_per_channel_'+tit+'.html'
  fig_path='../derivatives/sub-'+sid+'/megqc/figures/'+fig_name
  fig.write_html(fig_path)

  return(fig, fig_path)

  #figure out weird y axis, hovering is too long, simple slider

In [38]:
fig_m_pp_ampl_epoch, fig_path_m_pp_ampl_epoch=boxplot_std_channel_epoch_hovering_plotly(df_mg=df_pp_ampl_mags, tit='Magnetometers', sid='1')